In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import time
import numpy as np
import sqlite3

# Extract

In [2]:
bitcoin_path = 'data/bitcoin_price.csv'
dash_path = 'data/dash_price.csv'
ethereum_path = 'data/ethereum_price.csv'
iota_path = 'data/iota_price.csv'
litecoin_path = 'data/litecoin_price.csv'
monero_path = 'data/monero_price.csv'
neo_path = 'data/neo_price.csv'
nem_path = 'data/nem_price.csv'
numeraire_path = 'data/numeraire_price.csv'
omisego_path = 'data/omisego_price.csv'
qtum_path = 'data/qtum_price.csv'
ripple_path = 'data/ripple_price.csv'
stratis_path = 'data/stratis_price.csv'
waves_path = 'data/waves_price.csv'

bitcoin_df = pd.read_csv(bitcoin_path)
dash_df = pd.read_csv(dash_path)
ethereum_df = pd.read_csv(ethereum_path)
iota_df = pd.read_csv(iota_path)
litecoin_df = pd.read_csv(litecoin_path)
monero_df = pd.read_csv(monero_path)
neo_df = pd.read_csv(neo_path)
nem_df = pd.read_csv(nem_path)
numeraire_df = pd.read_csv(numeraire_path)
omisego_df = pd.read_csv(omisego_path)
qtum_df = pd.read_csv(qtum_path)
ripple_df = pd.read_csv(ripple_path)
stratis_df = pd.read_csv(stratis_path)
waves_df = pd.read_csv(waves_path)

sp_df = pd.read_csv('data/SP500data.csv')

# Transform

In [3]:
dataframes = [bitcoin_df, dash_df, ethereum_df, iota_df, litecoin_df, monero_df, neo_df, nem_df, numeraire_df, omisego_df, qtum_df, ripple_df, stratis_df, waves_df, sp_df]

names = ['bitcoin', 'dash', 'ethereum', 'iota', 'litecoin', 'monero', 'neo', 'nem', 'numeraire', 'omisego', 'qtum', 'ripple', 'stratis', 'waves', 'sp']

for name, frame in zip(names, dataframes):
    # The S&P dataframe is slightly different, so we treat it separately
    if name == 'sp':
        frame.drop('Adj Close', axis=1, inplace=True)
        frame.rename({'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True, axis=1)
        
    else:
        frame.rename({'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume', 'Market Cap': 'market_cap'}, inplace=True, axis=1)
        
        # Reformat date into YYYY-MM-DD
        frame['date'] = frame['date'].apply(lambda x: time.strftime('%Y-%m-%d', time.strptime(x, '%b %d, %Y')))
        
        # Replace '-' with NULL values
        frame['volume'] = frame['volume'].replace('-', np.NaN)
        
        frame.name = name

In [4]:
bitcoin_df.head()

,date,open,high,low,close,volume,market_cap
0,2018-02-20,11231.8,11958.5,11231.80,11403.7,"9,926,540,000","189,536,000,000"
1,2018-02-19,10552.6,11273.8,10513.20,11225.3,"7,652,090,000","178,055,000,000"
2,2018-02-18,11123.4,11349.8,10326.00,10551.8,"8,744,010,000","187,663,000,000"
3,2018-02-17,10207.5,11139.5,10149.40,11112.7,"8,660,880,000","172,191,000,000"
4,2018-02-16,10135.7,10324.1,9824.82,10233.9,"7,296,160,000","170,960,000,000"


# Load

In [5]:
db = 'cryptocurrency.sqlite'

engine = create_engine(f'sqlite:///{db}')

conn = sqlite3.connect(db)
c = conn.cursor()
with open('database_setup.sql', 'r') as script:
    qry = script.read()
    c.executescript(qry)
conn.commit()
conn.close()

In [6]:
engine.table_names()

['bitcoin',
 'dash',
 'ethereum',
 'iota',
 'litecoin',
 'monero',
 'nem',
 'neo',
 'numeraire',
 'omisego',
 'qtum',
 'ripple',
 'sp',
 'stratis',
 'waves']

In [7]:
for name, frame in zip(names, dataframes):
    frame.to_sql(name=name, con=engine, if_exists='replace', index=False)